In [30]:
from read_input2 import read_input
import math
import numpy
import time

## Lecture des données

In [31]:
start_time = time.time()

ville="Bordeaux" #  changer 
ressources,ressources_unavailabilities,tasks,tasks_unavailabilities = read_input(1,ville)
# "Bordeaux", "Finland", "GuineaGolf", "Italy", "Poland"

In [32]:
def dist(lat1, long1, lat2, long2):
    """
    fonction renvoyant la distance en kms entre deux points décrits par leur latitude et longitude
    """
    if lat1==lat2 and long1==long2:
        return 0
    return(numpy.arccos(numpy.sin(2*numpy.pi/360*lat1)*numpy.sin(2*numpy.pi/360*lat2)+numpy.cos(2*numpy.pi/360*lat1)*numpy.cos(2*numpy.pi/360*lat2)*numpy.cos(2*numpy.pi/360*(long2-long1)))*6371)

v=5/6

L_ressources = [j for j in ressources] # tableau des objets ressources 
L_tasks = [i for i in tasks] # tableau des objets taches
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (task.latitude, task.longitude) for task in tasks.values()}

dict_pos = {task : (tasks[L_tasks[task]].latitude, tasks[L_tasks[task]].longitude) for task in range(number_of_tasks)}
print(dict_pos)

#distance[a][b] donne la distance de la tâche a vers la tâche b
distances = [[dist(tasks[L_tasks[a]].latitude,tasks[L_tasks[a]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude) for b in range(number_of_tasks)] for a in range(number_of_tasks)]

#distance_m[j][i] donne la distance de la maison de j la tâche i
distances_m = [[dist(ressources[L_ressources[j]].latitude,ressources[L_ressources[j]].longitude, tasks[L_tasks[i]].latitude, tasks[L_tasks[i]].longitude) for i in range(number_of_tasks)] for j in range(number_of_ressources)]
print(distances_m)

{0: (44.556549383420084, -0.31939224223757195), 1: (44.967500952177986, -0.6086852638150881), 2: (45.14421541464031, -0.7342570469020379), 3: (45.264808304867096, -0.7717887212411139), 4: (45.044422793402624, -0.6687606009488057), 5: (45.19957452440505, -0.7462077931750715), 6: (45.397697776585, -0.9668192708194538), 7: (45.023479086796385, -0.8072126299796225), 8: (45.29291368453335, -0.9365361007032235), 9: (45.08146166752168, -0.8062453230620741)}
[[77.08814164242939, 26.425096734418076, 6.932258498761599, 13.231210250895991, 16.907606071243386, 8.018390527454564, 29.654865035592632, 14.251837565944708, 18.12703789005155, 7.855554978587318], [77.08814164242939, 26.425096734418076, 6.932258498761599, 13.231210250895991, 16.907606071243386, 8.018390527454564, 29.654865035592632, 14.251837565944708, 18.12703789005155, 7.855554978587318]]


## Création d'une solution initiale

In [33]:
def generate_time(task_list):
    """
    input : list of tasks indices
    output : list of integers
    generates a time table from a list of ordered tasks
    """
    time=[]
    time.append(tasks[L_tasks[task_list[0]]].openingTime)
    for i in range(1,len(task_list)):
        time.append(max((tasks[L_tasks[task_list[i]]].openingTime,time[-1]+distances[task_list[i]][task_list[i-1]]/v+tasks[L_tasks[task_list[i-1]]].duration)))
    return(time)



In [34]:
def employees_by_level():
    """
    output : list of employees ids from level 1 to level 3
    """
    employees=[]
    for i in range(number_of_ressources):
        if ressources[L_ressources[i]].level==1:
            employees.append(i)
    for i in range(number_of_ressources):
        if ressources[L_ressources[i]].level==2:
            employees.append(i)
    for i in range(number_of_ressources):
        if ressources[L_ressources[i]].level==3:
            employees.append(i)
    return(employees)


In [35]:
def tasks_by_level():
    """
    output : list of tasks ids from level 1 to level 3
    """
    ordered_tasks=[]
    for i in range(number_of_tasks):
        if tasks[L_tasks[i]].level==1:
            ordered_tasks.append(i)
    for i in range(number_of_tasks):
        if tasks[L_tasks[i]].level==2:
            ordered_tasks.append(i)
    for i in range(number_of_tasks):
        if tasks[L_tasks[i]].level==3:
            ordered_tasks.append(i)
    return(ordered_tasks)

In [36]:
def is_feasible(employee,task_list,hours_list):
    """
    input : employee id, employee task list (by id), employee hours list (by id)
    true if feasible
    # add tasks and ressources unavailabilities
    """
    if hours_list[-1]>ressources[L_ressources[employee]].workingEnd:
        return False
    return True

In [37]:
def initialize():
    """
    input :
    output :
    # level up the initial task attribution (by distance)
    # include worker skill
    """

    ordered_tasks = tasks_by_level()
    ordered_employees = employees_by_level()

    for t in ordered_tasks:
        unattributed_tasks=[]
        unattributed = True
        r=0
        while unattributed and r<len(ordered_employees):
            unattributed=False
            if ressources[L_ressources[ordered_employees[r]]].level>=tasks[L_tasks[t]].level:
                tasks_tmp = ressources[L_ressources[ordered_employees[r]]].tasks
                tasks_tmp.append(t)
                hours_tmp = generate_time(tasks_tmp)
                if is_feasible(r,tasks_tmp,hours_tmp):
                    ressources[L_ressources[ordered_employees[r]]].tasks = tasks_tmp
                    ressources[L_ressources[ordered_employees[r]]].hours = hours_tmp
                    unattributed=False
                r+=1
        if unattributed:
            unattributed_tasks.append(t)
    
    for r in ordered_employees:
        print(ressources[L_ressources[ordered_employees[r]]].tasks)
        print(ressources[L_ressources[ordered_employees[r]]].hours)
        assert len(ressources[L_ressources[ordered_employees[r]]].tasks) == len(ressources[L_ressources[ordered_employees[r]]].hours)

initialize()
        

[]
[]
[0, 1, 3, 6, 7, 9]
[480, 601.3031549906542, 703.843044594722, 789.320929159563, 901.459784695339, 969.1971645776409]


## Mutation

## Solution visualisation

In [ ]:
import plotly.express as px
import pandas as pd

# plotting the working tasks
liste=[]
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            start=int(s[j].x)
            end=(int(s[j].x)+tasks[L_tasks[j]].duration)
            liste.append(dict(Task=L_ressources[i], Start=start, Finish=end, Resource="executing task",J="tâche "+str(j+1)))

# plotting the ressource unavailabilities
for r in range(number_of_ressources):
    for R,id in runav_id:
        if r == R:
            start= ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityStart
            end= ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd
            liste.append(dict(Task=L_ressources[r], Start=start, Finish=end, Resource="unavailable",J="unavailability "+str(id+1)))

# plotting the transports
for a in range(number_of_tasks):
    for b in range(number_of_tasks):
        for j in range(number_of_ressources):
            if round(y[(j,a,b)].x):
                start=(int(s[a].x)+tasks[L_tasks[a]].duration)
                end=(start+distances[a][b]/vitesse)
                liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="moving",J="trajet "+str(a+1)+"-"+str(b+1)))

# plotting the first transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(d[(j,a)].x):
            start=(s[a].x-distances_m[j][a]/vitesse)
            end=s[a].x
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J="trajet maison -"+str(a+1)))

# plotting the last transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(f[(j,a)].x):
            start=(s[a].x + tasks[L_tasks[i]].duration)
            end=(start+ distances_m[j][a]/vitesse)
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J=str(a+1)+"- trajet maison"))

df = pd.DataFrame(liste)
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task",text="J")

fig.layout.xaxis.type = 'linear'
fig.data[0].x = df.delta.tolist()
fig.show()